In [6]:
import re
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as dt

from sklearn import model_selection

from sklearn.metrics import confusion_matrix

%matplotlib inline 

In [7]:
# all accepted loans from lendingclub
all_accept_df = pd.read_csv('./archive/accepted_2007_to_2018q4.csv')

C:\Users\Kalyan Manda\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
# drop invalid loan_status to 'Unknown'

all_accept_df.loc[pd.isnull(all_accept_df['loan_status']),'loan_status'] = 'Unknown'

all_accept_df.loan_status.unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default', 'Unknown',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [9]:
loan_status = all_accept_df['loan_status']

loan_status[(loan_status=='Fully Paid') | (loan_status=='Current') |
            (loan_status=='Does not meet the credit policy. Status:Fully Paid') |
            (loan_status=='In Grace Period')] = 1

loan_status[loan_status != 1] = 0

all_accept_df['loan_status'] = pd.to_numeric(loan_status)

<ipython-input-9-e98628443b49>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_status[(loan_status=='Fully Paid') | (loan_status=='Current') |
<ipython-input-9-e98628443b49>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_status[loan_status != 1] = 0


In [10]:
def ConvertDateTimeToOrdinal(d):
    if d is pd.NaT:
        return 0
    else:
        return d.toordinal()

In [11]:
# convert columns to datetime (toordinal)

dateCols = pd.Series(
    ['issue_d',
    'earliest_cr_line',
    'last_pymnt_d',
    'next_pymnt_d',
    'last_credit_pull_d',
    'debt_settlement_flag_date',
    'settlement_date',
    'hardship_start_date',
    'hardship_end_date',
    'payment_plan_start_date',
    'sec_app_earliest_cr_line'])

for col in dateCols:    
    all_accept_df[col] = pd.to_datetime(all_accept_df[col]).apply(ConvertDateTimeToOrdinal)

In [12]:
# drop columns to be ignore for now

all_accept_df = all_accept_df.drop([
    'id',
    'member_id',
    'emp_title',
    'emp_length',
    'home_ownership',
    'url',
    'desc',
    'title',
    'zip_code',
    'addr_state',
    'initial_list_status',
    'verification_status_joint',  
    'hardship_type',
    'hardship_reason',
    'disbursement_method'
], axis=1)

In [13]:
# convert column to numeric

all_accept_df['term'] = pd.to_numeric(all_accept_df['term'].apply(lambda x: re.findall(r'\d+', str(x))).str[0])
all_accept_df['deferral_term'] = pd.to_numeric(all_accept_df['deferral_term'])

In [14]:
# Dummify

colsToBeDummified = pd.Series(
    ['grade',
    'sub_grade',
    'verification_status',
    'purpose',
    'pymnt_plan',
    'application_type',
    'hardship_flag',
    'hardship_status',
    'hardship_loan_status',
    'debt_settlement_flag',
    'settlement_status'])

for col in colsToBeDummified:    
    all_accept_df = all_accept_df.join(pd.get_dummies(all_accept_df[col], drop_first=True, prefix=col))
    all_accept_df = all_accept_df.drop(col, axis=1)

In [15]:
# handle na

all_accept_df.fillna(0, inplace=True)

In [16]:
from sklearn.model_selection import train_test_split

all_accept_train_df, all_accept_test_df, all_accept_train_target, all_accept_test_target = \
            train_test_split(all_accept_df, all_accept_df['loan_status'], test_size=0.2, random_state=100)

In [17]:
# make sure all columns are numeric

for col in all_accept_train_df.columns:
    all_accept_train_df[col] = pd.to_numeric(all_accept_train_df[col])
    all_accept_test_df[col] = pd.to_numeric(all_accept_test_df[col])

<ipython-input-17-9b912882cf0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_accept_train_df[col] = pd.to_numeric(all_accept_train_df[col])
<ipython-input-17-9b912882cf0c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_accept_test_df[col] = pd.to_numeric(all_accept_test_df[col])


In [18]:
all_accept_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1808560 entries, 732625 to 1873416
Columns: 192 entries, loan_amnt to settlement_status_COMPLETE
dtypes: float64(113), int64(12), uint8(67)
memory usage: 1.8 GB


In [19]:
all_accept_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452141 entries, 71127 to 717237
Columns: 192 entries, loan_amnt to settlement_status_COMPLETE
dtypes: float64(113), int64(12), uint8(67)
memory usage: 463.5 MB


## - Feature Selection

In [20]:
features = [
    'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
    'int_rate', 'grade_B', 'grade_C',
    'grade_D', 'grade_E', 'grade_F', 'grade_G',
    'sub_grade_A2', 'sub_grade_A3', 'sub_grade_A4', 'sub_grade_A5', 
    'sub_grade_B2', 'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5',
    'sub_grade_C1', 'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4',
    'sub_grade_C5', 'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3',
    'sub_grade_D4', 'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2',
    'sub_grade_E3', 'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1',
    'sub_grade_F2', 'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5',
    'sub_grade_G1', 'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4',
    'sub_grade_G5', 'sub_grade_B1','purpose_credit_card', 'purpose_debt_consolidation',
    'purpose_home_improvement', 'purpose_medical', 'purpose_other', 
    'purpose_small_business', 'annual_inc',
    'issue_d', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high',
    'open_acc', 'total_acc', 'total_pymnt',
    'total_pymnt_inv', 'last_pymnt_d', 'last_pymnt_amnt',
    'annual_inc_joint', 'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
    'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
    'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high',
]

In [21]:
mask = all_accept_train_df.columns.isin(features)

kBestColumns = all_accept_train_df.columns[mask]
otherColumnsToBeCombined = all_accept_train_df.columns[~mask]

len(kBestColumns) + len(otherColumnsToBeCombined)

192

## - PCA on non-kBest columns

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)

### PCA - Modify train data set

In [23]:
all_accept_train_otherColsDf = pd.DataFrame(all_accept_train_df[otherColumnsToBeCombined])

data_rescaled = scaler.fit_transform(all_accept_train_otherColsDf)

pca.fit(data_rescaled)

reduced = pca.transform(data_rescaled)
reduced = pd.DataFrame(reduced)

reduced.fillna(0, inplace=True)

reduced.shape

(1808560, 21)

In [24]:
pd.DataFrame(all_accept_train_df[kBestColumns]).shape

(1808560, 75)

In [25]:
kBestColsDf = pd.DataFrame(all_accept_train_df[kBestColumns])

all_accept_train_df = \
    pd.concat([kBestColsDf, reduced.reindex(kBestColsDf.index)], axis=1)

all_accept_train_df.shape

(1808560, 96)

### PCA - Modify test data set

In [26]:
all_accept_test_otherColsDf = pd.DataFrame(all_accept_test_df[otherColumnsToBeCombined])

reduced = pca.transform(all_accept_test_otherColsDf)
reduced = pd.DataFrame(reduced)

reduced.fillna(0, inplace=True)

reduced.shape

(452141, 21)

In [27]:
kBestColsDf = pd.DataFrame(all_accept_test_df[kBestColumns])

all_accept_test_df = \
    pd.concat([kBestColsDf, reduced.reindex(kBestColsDf.index)], axis=1)

all_accept_test_df.shape

(452141, 96)

In [28]:
all_accept_train_df.shape[0]

1808560

## Shrink data set

In [29]:
n_range_train = range(1, all_accept_train_df.shape[0])
#n_range_train = range(1, 20000)

all_accept_train_df_r = pd.DataFrame(all_accept_train_df.iloc[n_range_train])
all_accept_train_target_r = all_accept_train_target.iloc[n_range_train] 

all_accept_train_df_r.fillna(0, inplace=True)

In [30]:
n_range_test = range(1, all_accept_test_df.shape[0])
#n_range_test = range(1, 2000)

all_accept_test_df_r = pd.DataFrame(all_accept_test_df.iloc[n_range_test])
all_accept_test_target_r = all_accept_test_target.iloc[n_range_test] 

all_accept_test_df_r.fillna(0, inplace=True)

## Global variables

In [31]:
random_state = 100
n_jobs = 6
#n_jobs = 2
cv = 5
accuracy = 'accuracy'

## Decision Trees

In [32]:
from sklearn import tree

tree_model = tree.DecisionTreeClassifier()

grid_para_tree = [{
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int)
}]

tree_model.set_params(random_state=random_state)

grid_search_tree = model_selection.GridSearchCV(tree_model, grid_para_tree,
                                                cv=cv, scoring=accuracy,
                                                n_jobs=n_jobs)

In [33]:
%time grid_search_tree.fit(all_accept_train_df_r, all_accept_train_target_r)

Wall time: 14h 3min 7s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=100), n_jobs=6,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'min_samples_leaf': range(1, 10),
                          'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30])}],
             scoring='accuracy')

In [34]:
grid_search_tree.score(all_accept_train_df_r, all_accept_train_target_r)

0.9916320120051378

In [35]:
grid_search_tree.score(all_accept_test_df_r, all_accept_test_target_r)

0.9868845932675719

In [36]:
confusion_matrix(grid_search_tree.predict(all_accept_train_df_r), all_accept_train_target_r)

array([[ 225034,    4009],
       [  11125, 1568391]], dtype=int64)

In [37]:
confusion_matrix(grid_search_tree.predict(all_accept_test_df_r), all_accept_test_target_r)

array([[ 55124,   2005],
       [  3925, 391086]], dtype=int64)

## XGBoost

In [38]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgb_c = xgb.XGBClassifier(
    objective= 'binary:logistic',
    nthread=n_jobs,
    seed=random_state
)

parameters = {
    'max_depth': range (5, 7, 10),
    'n_estimators': range(100, 150, 200),
    'learning_rate': [0.25, 0.2, 0.1]
}

grid_search_xgb = model_selection.GridSearchCV(
    estimator=xgb_c,
    param_grid=parameters,
    scoring = accuracy,
    n_jobs = n_jobs,
    cv = cv
)

In [39]:
%time grid_search_xgb.fit(all_accept_train_df_r, all_accept_train_target_r)

C:\Users\Kalyan Manda\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:50:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 47min 42s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None, nthread=6,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, seed=100,
                                     subsample=No

In [40]:
grid_search_xgb.score(all_accept_train_df_r, all_accept_train_target_r)

0.9917414914304703

In [41]:
grid_search_xgb.score(all_accept_test_df_r, all_accept_test_target_r)

0.9911797230946167

In [42]:
confusion_matrix(grid_search_xgb.predict(all_accept_train_df_r), all_accept_train_target_r)

array([[ 221916,     693],
       [  14243, 1571707]], dtype=int64)

In [43]:
confusion_matrix(grid_search_xgb.predict(all_accept_test_df_r), all_accept_test_target_r)

array([[ 55264,    203],
       [  3785, 392888]], dtype=int64)

## Neural Network

In [44]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import sklearn
import time
import scipy

from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import initializers

In [45]:
class One_layer_batch(object):
    kernel_initializer = initializers.TruncatedNormal(mean=0.0,stddev=0.1)
    bias_initializer   = initializers.Constant(0.1)    
    def __init__(self, size_hidden=2):
        self.__w1 = None
        self.__b1 = None
        self.__w2 = None
        self.__b1 = None
        self.__size_hidden = size_hidden
        
    def __initialize_model(self, x_train, y_train, rate):
        num_observations= x_train.shape[0]
        num_features   = x_train.shape[1]
        num_labels     = y_train.shape[1]
        size_hidden    = self.__size_hidden
        
        x = tf.placeholder(tf.float32, shape=[None, num_features])
        y = tf.placeholder(tf.float32, shape=[None, num_labels])
        
        d_layer1 = Dense(input_shape=(num_features,), units=size_hidden, activation=tf.nn.sigmoid, kernel_initializer = self.kernel_initializer, bias_initializer= self.bias_initializer)
        hidden   = d_layer1(x)
        
        hidden_drop = tf.nn.dropout(hidden, rate=rate)
        
        d_layer2 = Dense(input_shape=(size_hidden,), units=num_labels, kernel_initializer = self.kernel_initializer, bias_initializer= self.bias_initializer)
        y_lin   = d_layer2(hidden_drop)

        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_lin, labels=y))
        
        train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
        return train_step, x, y, d_layer1.weights[0], d_layer1.weights[1], d_layer2.weights[0], d_layer2.weights[1]
        
    def fit(self, x_train, y_train, rate = 0.25, batch_size = 100, steps = 10000):
        train_step, x, y, w_1, b_1, w_2, b_2 = self.__initialize_model(x_train, y_train, rate)
        
        num_observations = x_train.shape[0]
        tf.global_variables_initializer().run()
        
        for i in range(steps):
            locs    = sklearn.utils.random.sample_without_replacement(num_observations, batch_size)
            x_batch = x_train.values[locs,:]
            y_batch = y_train[locs,:]
            train_step.run(feed_dict={x: x_batch, y: y_batch})
            
        self.__w1 = w_1.eval()
        self.__b1 = b_1.eval()
        self.__w2 = w_2.eval()
        self.__b2 = b_2.eval()
    
    def __sigmoid(self, x):
        return scipy.special.expit(x)
        
    def predict(self, x_train):
        w1 = self.__w1
        b1 = self.__b1
        w2 = self.__w2
        b2 = self.__b2
        y_lin = np.dot(self.__sigmoid(np.dot(x_train, w1)+b1), w2)+b2
        return tf.argmax(y_lin, 1).eval()        
    
    @property
    def w1(self):
        return self.__w1
    
    @property
    def b1(self):
        return self.__b1
    
    @property
    def w2(self):
        return self.__w2
    
    @property
    def b2(self):
        return self.__b2

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [46]:
start = time.time()
sess = tf.InteractiveSession()
tf.set_random_seed(random_state)

all_accept_train_target_r_dum = pd.get_dummies(pd.Series(all_accept_train_target_r)).values
all_accept_test_target_r_dum = pd.get_dummies(pd.Series(all_accept_test_target_r)).values

nn = One_layer_batch(size_hidden=40)
nn.fit(all_accept_train_df_r, all_accept_train_target_r_dum, batch_size = 100, steps = 50000)

predict = nn.predict(all_accept_train_df_r)
print ('Training Accuracy: ', np.mean(predict == np.argmax(all_accept_train_target_r_dum, 1)))

predict = nn.predict(all_accept_test_df_r)
print('Test Accuracy: ', np.mean(predict == np.argmax(all_accept_test_target_r_dum, 1)))

print('Time elapsed: ', time.time()-start)

sess.close()

Training Accuracy:  0.8694214565297566
Test Accuracy:  0.8694010704649002
Time elapsed:  32186.697366952896
